## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [273]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [274]:
class MyDecisionTreeClassifier(DecisionTreeClassifier):
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        # min_samples_split - The minimum number of samples required to split an internal node
        # max_depth - The maximum depth of the tree
        # max_features - The number of features to consider when looking for the best split:
        #    If int, then consider max_features features at each split.
        #    If float, then max_features is a percentage and int(max_features * n_features) features are considered at each split.
        #    If “sqrt”, then max_features=sqrt(n_features).
        #    If “log2”, then max_features=log2(n_features).
        #    If None, then max_features=n_features.
        # sufficient_share - ?
        # 

        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth 
        self.sufficient_share = sufficient_share 
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
    
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float').reshape(1,-1)
        r_s = r_s.astype('float').reshape(1,-1)
        # Здесь пытался посчитать в лоб, по формуле
        #parent_size = l_s[0][0] + r_s[0][0]
        #gini_l = 1 - np.sum((l_c/l_s)**2, axis=1)
        #gini_r = 1 - np.sum((r_c/r_s)**2, axis=1)
        # Здесь чуть-чуть оптимизировал, но выигрыша в скорости это не дало :(
        gini_l = l_s - np.sum(l_c**2, axis=1)/l_s
        gini_r = r_s - np.sum(r_c**2, axis=1)/r_s
        gini = gini_l.reshape(-1,1) + gini_r.reshape(-1,1)
        return gini
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        # Здесь считаю в лоб
        parent_size = l_s[0][0] + r_s[0][0]
        entropy_l = np.sum((l_c/l_s)*np.log2(l_c/l_s), axis=1)
        entropy_r = np.sum((r_c/r_s)*np.log2(r_c/r_s), axis=1)
        entropy = (l_s/parent_size)*entropy_l.reshape(-1,1) + (r_s/parent_size)*entropy_r.reshape(-1,1)
        return entropy

    def __misclass(self, l_c, l_s, r_c, r_s):
        # Здесь считаю в лоб
        parent_size = l_s[0][0] + r_s[0][0]
        mc_l = 1 - np.max(l_c/l_s, axis=1)
        mc_r = 1 - np.max(r_c/r_s, axis=1)
        mc = (l_s/parent_size)*mc_l.reshape(-1,1) + (r_s/parent_size)*mc_r.reshape(-1,1)
        return mc

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:round(np.sqrt(n_feature)).astype('int')]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:round(np.log2(n_feature)).astype('int')]

    def __get_feature_ids_N(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort() # массив индексов отсортированного массива x
        return x[sorted_idx], y[sorted_idx] # отсортированный массив х, и, соответствующий ему, массив y 

    def __div_samples(self, x, y, feature_id, threshold):
        # if x[i:feature_id]<threshold: left_mask[i] = True, else left_mask[i] = False
        left_mask = x[:, feature_id] <= threshold 
        # инвертируем left_mask 
        right_mask = ~left_mask 
        # делим матрицу x и вектор y в соответствии с массивами left_mask и right_mask 
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask] 
    

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # 1. Сортируем вектор признака - x и, соответствующий ему, вектор целевой переменной - y
        # 2. Сохраняем в class_number кол-во классов (кол-во уникальных значений вектора целевой переменной - y)
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # 1. Выбрасываем из вектора sorted_y по краям min_samples_split-1 элементов, так как 
        # нет смысла проверять точки разделеня признака, если в одном из детей, 
        # образованных после потенциального разделении, будет меньше, чем min_samples_split наблюдений.
        # В оригинальном коде ДЗ было sorted_y[self.min_samples_split:-self.min_samples_split], но в этом случае
        # алгоритм не будет рассматривать разбиения лежащие между индексами min_samples_split-1 и min_samples_split.
        # Пример: 0 0 1 1 1 1 0 0. При min_samples_split=2. В оригинале разбиений между 1 и 2 индексом и 
        # между 5 и 6 индексом не будет, хотя, по логике они должны быть.
        # Поэтому рассматриваем интервал min_samples_split-1:-min_samples_split+1.
        # 2. Определяем индексы элементов, на которых происходит смена целевого класса в векторе splitted_sorted_y.
        # Соответственно, это те точки, перед которыми надо будет проводить потенциальное разделение и считать IG. 
        splitted_sorted_y = sorted_y[self.min_samples_split-1:-self.min_samples_split+1]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split - 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # 1. Подсчитывает по разности индексов кол-во наблюдений между сменами целевого класса (текущий индекс изменения
        # целевого класса минус предыдущий индекс изменения целевого класса)
        # 2. Вводим вспомогательную матрицу one_hot_code для получения кол-ва наблюдений по классам при разных разбиениях
        # 3. Кол-во строк в one_hot_code равняется кол-ву возможных разбиений, а кол-во столбцов = кол-ву целевых классов.  
        # Элемент one_hot_code[i][j] = 1, если при (i+1)-ом возможном разбиении мы переходим от j целевого класса
        # ко второму целевому классу (в случаии двух целевых классов)
        # 4. Матрица class_increments содержит кол-во элементов соответствующего целевого класса при возможных разбиениях. 
        # Элемент class_increments[i][j] = k, если при (i+1)-ом возможном разбиении кол-во элементов c целевым классом j = k
        # 5. class_increments[0] - соответсвует кол-ву наблюдений при первом разбиении по целевым классам. 
        # Так как мы не учитывали значения целевой переменной для первых min_samples_split наблюдений, то сейчас это надо сделать.
        eq_el_count = r_border_ids - np.append([self.min_samples_split-1], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), splitted_sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # 1. l_class_count - кол-во наблюдений, попавших в левую часть после разбиения 
        # по возможным разбиениям (строчки) и по целевым классам (столбцы)
        # 2. r_class_count - кол-во наблюдений, попавших в правую часть после разбиения 
        # по возможным разбиениям (строчки) и по целевым классам (столбцы)
        # 3. l_sizes - кол-во наблюдений, попавших в левую часть после разбиения, по всем возможным разбиением (строки)
        # 4. r_sizes - кол-во наблюдений, попавших в правую часть после разбиения, по всем возможным разбиением (строки)
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1) + 1
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # 1. Считаем выбранную функцию impurity. gs - вектор impurity по возможным разбиениям.
        # 2. Выбираем номер разбиения, на котором имеем минимум impurity  
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # 1. Получаем индекс точки разбиения, на котором impurity достигает минимального значения
        # 2. Возвращаем минимальную impurity и порог разделения
        left_el_id = l_sizes[idx][0]
        return gs[idx][0], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Проверяем условия остановки
        # Условие №1. Максимальное кол-во в вершине наблюдений одного из целевых класса 
        # Условие №2. Глубина дерева достигла максимального значения
            
        feature_ids = self.get_feature_ids(x.shape[1])

        if np.bincount(y).max() >= self.sufficient_share*y.size or (self.max_depth is not None and depth >= self.max_depth) or \
           y.size == self.min_samples_split or feature_ids.size == 0:              
            node = [self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).max()/y.size]
            return node
        
        impurity_min = float('+inf')
        node = None
        for idx in feature_ids:  
            impurity = self.__find_threshold(x[:,idx], y)
            if impurity[0] < impurity_min:
                impurity_min = impurity[0]
                threshold = impurity[1]
                feature_id = idx
                node = [self.NON_LEAF_TYPE, feature_id, threshold]
                
        if node is not None:        
            x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)  
            self.tree[2 * node_id + 1] = self.__fit_node(np.delete(x_l, feature_id, 1), y_l, 2*node_id+1, 0, depth+1)
            self.tree[2 * node_id + 2] = self.__fit_node(np.delete(x_l, feature_id, 1), y_l, 2*node_id+2, 0, depth+1)    
        else: node = [self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).max()/y.size]
        return node
       
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE +
        # self.NON_LEAF_TYPE +

        # self.tree +
        # self.max_depth +
        # self.sufficient_share +
        # self.min_samples_split +

        # self.get_feature_ids +
        # self.__find_threshold +
        # self.__div_samples + 
        # self.__fit_node +
        
        # return self.tree
        #pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.tree[0] = self.__fit_node(x, y, 0, 0) 
  
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node 
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [38]:
# Тут разбор кода. Смотреть не надо
x = np.array([[4,1,3,0,2],[5,9,2,1,6],[8,3,6,1,9],[0,4,2,6,7],[9,4,1,3,0],[6,2,6,7,2],[2,8,2,5,9],[3,1,6,0,2]])
y = np.array([1,0,1,1,1,1,0,0])
min_samples_split = 2
print(x,y)
feature = x[:,1]
print(feature)
sorted_idx = feature.argsort()
sorted_x, sorted_y = feature[sorted_idx], y[sorted_idx]
print("sorted_x=", sorted_x)
print("sorted_y=", sorted_y)
class_number = np.unique(y).shape[0]
print("class_number=",class_number)
splitted_sorted_y = sorted_y[min_samples_split-1:-min_samples_split+1]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (min_samples_split - 1)
print("splitted_sorted_y=", splitted_sorted_y)
print("r_border_ids=", r_border_ids)

eq_el_count = r_border_ids - np.append([min_samples_split-1], r_border_ids[:-1])
print("eq_el_count=",eq_el_count)
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
print("one_hot_code=",one_hot_code)
print("sorted_y[r_border_ids - 1]=",sorted_y[r_border_ids - 1])
one_hot_code[np.arange(r_border_ids.shape[0]), splitted_sorted_y[r_border_ids - 1]] = 1
print("one_hot_code=",one_hot_code)
print("eq_el_count.reshape(-1, 1)=",eq_el_count.reshape(-1, 1))
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
print("class_increments=",class_increments)
class_increments[0] = class_increments[0] + np.bincount(sorted_y[:min_samples_split], minlength=class_number)
print(np.bincount(sorted_y[:min_samples_split], minlength=class_number))
print("class_increments=",class_increments)

l_class_count = np.cumsum(class_increments, axis=0)
print('l_class_count=',l_class_count)
r_class_count = np.bincount(y) - l_class_count
print('r_class_count=',r_class_count)
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1) + 1
print('l_sizes=',l_sizes)
r_sizes = sorted_y.shape[0] - l_sizes
print('r_sizes=',r_sizes)

def __gini(l_c, l_s, r_c, r_s):
    l_s = l_s.astype('float')
    r_s = r_s.astype('float')
    parent_size = l_s[0][0] + r_s[0][0]
    print("parent_size=",parent_size)
    gini_l = 1 - np.sum((l_c/l_s)**2, axis=1)
    print("gini_l=",gini_l)
    gini_r = 1 - np.sum((r_c/r_s)**2, axis=1)
    print("gini_r=",gini_r)
    print("l_s/parent_size=",l_s/parent_size)
    print("(l_s/parent_size)*gini_l=",(l_s/parent_size)*gini_l.reshape(-1,1))
    gini = (l_s/parent_size)*gini_l.reshape(-1,1) + (r_s/parent_size)*gini_r.reshape(-1,1)
    return gini
gs = __gini(l_class_count, l_sizes, r_class_count, r_sizes)
print("gs=", gs)
idx = np.argmin(gs)
print ("idx=", idx)
left_el_id = l_sizes[idx][0]
print("left_el_id=",left_el_id)
print("gs[idx]=",gs[idx])
print((sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0)
print(x[:,0])

[[4 1 3 0 2]
 [5 9 2 1 6]
 [8 3 6 1 9]
 [0 4 2 6 7]
 [9 4 1 3 0]
 [6 2 6 7 2]
 [2 8 2 5 9]
 [3 1 6 0 2]] [1 0 1 1 1 1 0 0]
[1 9 3 4 4 2 8 1]
sorted_x= [1 1 2 3 4 4 8 9]
sorted_y= [1 0 1 1 1 1 0 0]
class_number= 2
splitted_sorted_y= [0 1 1 1 1 0]
r_border_ids= [1 5]
eq_el_count= [0 4]
one_hot_code= [[ 0.  0.]
 [ 0.  0.]]
sorted_y[r_border_ids - 1]= [1 1]
one_hot_code= [[ 1.  0.]
 [ 0.  1.]]
eq_el_count.reshape(-1, 1)= [[0]
 [4]]
class_increments= [[ 0.  0.]
 [ 0.  4.]]
[1 1]
class_increments= [[ 1.  1.]
 [ 0.  4.]]
l_class_count= [[ 1.  1.]
 [ 1.  5.]]
r_class_count= [[ 2.  4.]
 [ 2.  0.]]
l_sizes= [[2]
 [6]]
r_sizes= [[6]
 [2]]
parent_size= 8.0
gini_l= [ 0.5         0.27777778]
gini_r= [ 0.44444444  0.        ]
l_s/parent_size= [[ 0.25]
 [ 0.75]]
(l_s/parent_size)*gini_l= [[ 0.125     ]
 [ 0.20833333]]
gs= [[ 0.45833333]
 [ 0.20833333]]
idx= 1
left_el_id= 6
gs[idx]= [ 0.20833333]
6.0
[4 5 8 0 9 6 2 3]


In [42]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120269 entries, 1 to 150000
Data columns (total 11 columns):
SeriousDlqin2yrs                        120269 non-null int64
RevolvingUtilizationOfUnsecuredLines    120269 non-null float64
age                                     120269 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    120269 non-null int64
DebtRatio                               120269 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         120269 non-null int64
NumberOfTimes90DaysLate                 120269 non-null int64
NumberRealEstateLoansOrLines            120269 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    120269 non-null int64
NumberOfDependents                      120269 non-null float64
dtypes: float64(4), int64(7)
memory usage: 11.0 MB
None


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [94]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0]).astype('int32')
#np.unique(df["SeriousDlqin2yrs"])

In [107]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [110]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

12.573514223098755
2.200439929962158


In [ ]:
# Со скоростью проблемы! Не очень понимаю, где можно оптимизировать

## Проверка качества работы

In [90]:
gkf = KFold(n_splits=5, shuffle=True)

In [102]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]   
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9307391702
0.928244782573
0.931778498379
0.929658268895
0.932149835779


In [103]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.89394695269
0.889082896815
0.892450320113
0.891535711316
0.889327734586


In [ ]:
# Зато с качеством чуть лучше, чем у sklearn

# Применить для задачи Titanic 

In [112]:
df = pd.read_csv('train.csv', na_values='NaN')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [162]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [179]:
def drop_columns(df):
    return df.drop(labels = ['PassengerId', 'Embarked', 'Cabin', 'Name', 'Ticket'], axis=1)

def get_sex(df):
    df.set_value(df.Sex == 'male','Sex',1)
    df.set_value(df.Sex == 'female','Sex',0)
    return df

def split_sibsp(df):   
    Sib = np.zeros(len(df.SibSp))
    Sp = np.zeros(len(df.SibSp))
    for passenger in df.iterrows():    
        if passenger[1]['SibSp']>=2: 
            Sib[passenger[0]]=passenger[1]['SibSp']
            continue

        if (passenger[1]['SibSp']==1) and (passenger[1]['Sex']=='female') and (passenger[1]['Name'].find('Miss')!=-1):
            Sib[passenger[0]]=1
            continue        

        if (passenger[1]['SibSp']==1) and (passenger[1]['Sex']=='female') and (passenger[1]['Name'].find('Mrs')!=-1):
            surname = passenger[1]['Name'].split(',')[0]
            Find = False
            for pas in df.iterrows():
                if (pas[1]['Name'].startswith(surname + ',')) and (pas[1]['SibSp']==1) and \
                   (pas[1]['Name'].find('Mr')!=-1) and (pas[1]['Ticket']==passenger[1]['Ticket']):
                    Sp[passenger[0]] = 1
                    Find = True
                    break
            if Find is not True: Sib[passenger[0]] = 1
            continue

        if (passenger[1]['SibSp']==1) and (passenger[1]['Sex']=='male') and (passenger[1]['Name'].find('Mr')!=-1):
            surname = passenger[1]['Name'].split(',')[0]
            Find = False
            for pas in df.iterrows():
                if (pas[1]['Name'].startswith(surname + ',')) and (pas[1]['SibSp']==1) and \
                   (pas[1]['Name'].find('Mrs')!=-1) and (pas[1]['Ticket']==passenger[1]['Ticket']):
                    Sp[passenger[0]] = 1
                    Find = True
                    break
            if Find is not True: Sib[passenger[0]] = 1   
            continue

    df['Sib'] = Sib
    df['Sp'] = Sp
    return df

def set_age(df1):
    child_mean = int(df1.loc[df1.Age<=12]['Age'].mean())
    adult_mean = int(df1.loc[df1.Age>12]['Age'].mean())
    
    for passenger in df1.loc[pd.isnull(df1['Age'])].iterrows():
        surname = passenger[1]['Name'].split(',')[0]    
        if passenger[1]['Sib']>0:     
            namesakes_age = []
            for pas in df1.iterrows():         
                if (pas[1]['Name'].startswith(surname + ',')) and \
                   (pas[1]['Sib']==passenger[1]['Sib']) and \
                   (pas[1]['Ticket']==passenger[1]['Ticket']) and \
                   (pd.isnull(pas[1]['Age']) is not True): 
                        namesakes_age.append(pas[1]['Age'])

            if len(namesakes_age)>0: df1.set_value(passenger[0],'Age', int(np.array(namesakes_age).mean())) 
            else: df1.set_value(passenger[0],'Age', child_mean)

        if passenger[1]['Sp']>0:
            for pas in df1.iterrows(): 
                if (pas[1]['Name'].startswith(surname + ',')) and (pas[1]['Sp']==passenger[1]['Sp']) and \
                   (pas[1]['Ticket']==passenger[1]['Ticket']) and (isinstance(pas[1]['Age'], float)):
                        df1.set_value(passenger[0],'Age', pas[1]['Age'])
                else: df1.set_value(passenger[0],'Age', adult_mean)

        if passenger[1]['Sp']==0 and passenger[1]['Sib']==0: 
            df1.set_value(passenger[0],'Age', adult_mean)
    return df1

def drop_columns2(df):
    return df.drop(labels = ['Sib', 'Sp'], axis=1)

In [182]:
pipeline = make_pipeline(FunctionTransformer(split_sibsp, validate=False), \
                         FunctionTransformer(set_age, validate=False), \
                         FunctionTransformer(get_sex, validate=False), \
                         FunctionTransformer(drop_columns, validate=False), \
                         FunctionTransformer(drop_columns2, validate=False))

In [183]:
dff = pipeline.fit_transform(df)
dff.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [184]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int64(4), object(1)
memory usage: 45.3+ KB


In [243]:
x = dff.as_matrix(columns=dff.columns[1:])
y = dff.as_matrix(columns=dff.columns[:1])
y = y.reshape(y.shape[0]).astype('int32')

In [332]:
# Таким вот идиотским способом реализовал подбор параметров по сетке, так как RandomizeSearchCV и GridSearchCV 
# требуют в качестве estimator'а объект sklearn. А при наследовании от DecisionTreeClassifier ругается на параметр criterion
# при инициации.
n_splits = 5
gkf = KFold(n_splits=n_splits, shuffle=True)
all_accuracy = []
for min_samples_split in range(1,4):
    for max_depth in range(2,11):
        accuracy = []
        for train, test in gkf.split(x, y):
            X_train, y_train = x[train], y[train]
            X_test, y_test = x[test], y[test] 
            my_clf = MyDecisionTreeClassifier(min_samples_split=min_samples_split, max_depth=max_depth)
            my_clf.fit(X_train, y_train)
            accuracy.append(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))
        all_accuracy.append([np.mean(accuracy),min_samples_split,max_depth])

In [334]:
error, min_samples_split, max_depth = all_accuracy[np.argmax(all_accuracy, axis=0)[0]]
print("error =", error, "min_samples_split =", min_samples_split, "max_depth =", max_depth)

error = 0.616226225598 min_samples_split = 1 max_depth = 7
